# 하드웨어해킹 - Day01

## LED 켜기

In [3]:
V = 9.0

In [4]:
R = 1800

In [5]:
I = V/R
I

0.005

- ~: 교류
- --: 직류
- 통전 테스트: 리드봉이 전기적으로 연결됐을 때 스피커에 소리가 남

## 혀로 체험하기

In [7]:
V = 9.0

In [8]:
R = 50000

In [9]:
I = V/R
I

0.00018

## mAh?

- 밀리엄페어 시
- 전하의 용량을 표시한는 단위, 주로 배터리의 단위 표시

## 정전압 어댑터의 용량

- 전압: 일정 ex> 항상 5v
- 전류: 제품차이 ex) 2A(=2000mA) <= 최대 2A 공급 가능

# 3. AVR 프로그래밍

## 개요

- 주변에 산재한 전자장비들에 대한 분석능력 함양
  - 유무선공유기, IPTV, 홈 네트워크, 스마트폰, 전자사전
  - 구성요소와 작동방식의 이해도 향상
- 원하는 제품/Tool을 직접 제작할 수 있는 능력 개발
  - 장난감, 아이디어 제품
  - 하드웨어 기반 해킹도구
    - Eletronic Sniffer
    - Firmware Dumper
    - JTAG

## 마이크로 컨트롤러 소개

- Micro Controller Unit
- 주로 C언어를 이용하여 프로그래밍
- 칩 내부에 RAM과 ROM 등을 포함
  - System on a chip(SoC)

## MCU의 구조

- CPU, FLASH, SRAM, ROM, I/O
- 하나의 칩에 모든 기본 기능이 내장됨
- MCU 하나만으로 동작 가능

## AVR의 장점

- AVR은 하드웨어 프로그래밍 분야의 C언어이다!
  - 가장 먼저, 그리고 필수적으로 학습하기를 추천
  - Low Level의 지식들을 학습할 수 있음

## Atmega128A 소개

- RISC 아키텍쳐 기반
- 8비트 마이크로 컨트롤러
- 최대 16Mhz의 속도
- 128Kbytes의 플래쉬 메모리 내장
- 4Kbytes의 SRAM 내장

- 3.3v, 5v, 12v의 차이점
- 3.3v 어댑터를 쓸 수 있느냐? 없느냐?
- 독립적으로 작동하느냐? 아니면 통신을 해야되는 상황이 오게되면 기준 전압이 맞기 때문에 통신이 된다.
- 주변장치와 통신할 때 장점

## LED DOT MATRIX

## 입출력 포트의 이해

- 하드웨어 분야에서 포트란, 전기적 신호 데이터가 오가는 통로를 의미
- Atmega128
  - 총 64개의 핀 노출
  - 총 7개의 포트
  - Port A: PA0 ~ PA7
  - [Atmega128A Data sheet](http://www.atmel.com/Images/Atmel-8151-8-bit-AVR-ATmega128A_Datasheet.pdf)

- ddrB가 00 이 되면 안됨 0xff 이어야 함
- DDRA = 0xff; DDRB = 0xff;

### PA3번만 LED 점등

```c
#include <avr/io.h>

int main(void)
{
	DDRA = 0xff;
	//DDRA = 0b11111111;
	PORTA = 0b00001000;
	
	return 0;
}
```

### 1초씩 점등

```c
#define F_CPU 8000000UL
#include <avr/io.h>
#include <util/delay.h>

int main(void)
{
	DDRA = 0xff;
	while(1)
	{
		PORTA = 0xff; 
		_delay_ms(1000); 
		PORTA = 0x00; 
		_delay_ms(1000);
	} 
}
```

### 홀수번째 LED 켜기

```c
#include <avr/io.h>

int main(void)
{
	DDRA = 0xff;
	//DDRA = 0b11111111;
	//PORTA = 0b00001000;
	PORTA = 0b01010101;
	
	return 0;
}
```

## 스위치

- 왼쪽으로 했을 때 켜짐 가정
- 아무것도 넣지 않으면 0V 겠네 라고 생각할 수 있는데...
- 입력핀에는 5V를 확실히 넣어주거나 0V를 확실히 넣어줘야 됨


- 오늘 책 사서 봐야할 것은?

```c
#include <avr/io.h>

int main(void)
{
    int Input = 0;
    DDRA = 0xff;
    DDRB = 0x00;
    DDRC = 0xff;
    
    PORTC = 0xff;
    
    while(1)
    {
        Input = PINB;
        if(Input & 0x01)
            PORTA = 0xff;
        else
            PORTA = 0x00;
    }
}
```

## UART 프로그래밍

- 통신 프로토콜 중 하나
- USART(Universal Synchronous and Asynchronous Receiver/Transmitter)
- 특히 디버깅 메시지 출력 용도로 많이 쓰임

- ISP, UART 차이점?
- ISP: In-System Programming: 플래쉬롬을 탈착하지 않고 프로그램 기록 가능한 형태를 의미

```c
#define F_CPU 8000000UL 
#include <avr/io.h> 
#include <util/delay.h>

int main(void)
{
	/* Status Register 0A */ 
	UCSR0A = 0x00;
	
	/* Status Register 0B */
	/* RX/TX Enable = 10011000 */ 
	UCSR0B = 0x98;
	
	/* Status Register 0C */
	/* No parity, 8bit = 0110 */ 
	UCSR0C = 0x06;
	
	/* 중요 : Baud Rate 설정 */ /* BPS = 9600 */
	UBRR0H = 0;
	UBRR0L = 47;
	while(1)
	{
		UDR0 = 'A';
		_delay_ms(1000); 
	}
}
```

- UCSR: Control and Status Register
- UART: 소켓을 왜 쓰느냐는 질문과 같다.